In [152]:
from mxnet import autograd,nd
import pandas as pd
import numpy as np
data = pd.read_csv("./ch1/train.csv")
arr = np.array(data)
arr[:4999,3:6]

array([[281.0, 281.0, 281.0],
       [283.2, 283.2, 283.2],
       [281.4, 281.4, 281.4],
       ...,
       [1538.5, 1553.1, 1532.6],
       [1553.0, 1561.2, 1546.0],
       [1557.4, 1557.7, 1541.3]], dtype=object)

In [153]:
features = arr[:4999,3:]
features = nd.array(features)
features


[[ 281.   281.   281.   282.7    4. ]
 [ 283.2  283.2  283.2  281.1   16. ]
 [ 281.4  281.4  281.4  281.4    0. ]
 ...
 [1538.5 1553.1 1532.6 1561.   238. ]
 [1553.  1561.2 1546.  1563.7  291. ]
 [1557.4 1557.7 1541.3 1556.2  163. ]]
<NDArray 4999x5 @cpu(0)>

In [154]:
labels = arr[1:,6:7]
labels = nd.array(labels)
labels = labels.reshape(shape=[4999])
labels


[ 281.1  281.4  281.9 ... 1563.7 1556.2 1544. ]
<NDArray 4999 @cpu(0)>

In [155]:
from mxnet.gluon import data as gdata

# 超参数
batch_size = 10
dataset = gdata.ArrayDataset(features,labels)
data_iter = gdata.DataLoader(dataset,batch_size,shuffle=True)

for X,y in data_iter:
    print(X,y)
    break


[[ 573.   573.   573.   573.     2. ]
 [1317.4 1317.4 1317.4 1317.4   10. ]
 [ 992.5 1009.   992.5 1008.  1393. ]
 [1064.  1068.4 1056.5 1064.2  183. ]
 [ 802.   802.   780.   787.1   29. ]
 [1415.6 1415.6 1415.6 1415.6  110. ]
 [1213.7 1214.7 1195.8 1322.9  521. ]
 [1348.8 1348.8 1348.8 1348.8   37. ]
 [1235.9 1235.9 1235.8 1359.3   73. ]
 [ 277.   277.   277.   277.     0. ]]
<NDArray 10x5 @cpu(0)> 
[ 586.7 1318.5  999.5 1063.9  757.9 1414.  1295.9 1344.9 1361.3  275.8]
<NDArray 10 @cpu(0)>


In [156]:
from mxnet.gluon import nn
net = nn.Sequential()
net.add(nn.Dense(1))

In [157]:
from mxnet import init
net.initialize(init.Normal(sigma=0.01))

In [158]:
from mxnet.gluon import loss as gloss
loss = gloss.L2Loss()

In [159]:
from mxnet import gluon
#第一个超参数，学习率，并不是越小越好
trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.000000001})

In [160]:
test_data = np.array(pd.read_csv("./ch1/test.csv"))
test = nd.array(test_data[0:690,3:])
true_y = test_data[1:691,6:7]
true_y = nd.array(true_y)

In [161]:
def test_acc():
    return loss(net(test),true_y).mean().asnumpy()

In [162]:
num_epochs = 10
for epoch in range(1,num_epochs+1):
    for X,y in data_iter:
        with autograd.record():
            l = loss(net(X),y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features),labels)
    print('epoch %d,loss: %f test_lost: %f' % (epoch,l.mean().asnumpy(),test_acc()))

epoch 1,loss: 7556.174316 test_lost: 27591.605469
epoch 2,loss: 351.429443 test_lost: 521.468689
epoch 3,loss: 258.950500 test_lost: 297.154633
epoch 4,loss: 257.274231 test_lost: 242.258499
epoch 5,loss: 257.156067 test_lost: 243.840790
epoch 6,loss: 257.158813 test_lost: 248.009537
epoch 7,loss: 257.226379 test_lost: 246.651138
epoch 8,loss: 257.017609 test_lost: 243.123688
epoch 9,loss: 256.991516 test_lost: 244.455963
epoch 10,loss: 257.643982 test_lost: 283.432007


In [163]:
dense = net[0]
dense.weight.data()


[[ 2.7701187e-01  2.4466503e-01  2.3281887e-01  2.4825041e-01
  -1.7844135e-04]]
<NDArray 1x5 @cpu(0)>

In [164]:
dense.bias.data()


[0.00021783]
<NDArray 1 @cpu(0)>